In [1]:
import os
print(os.getcwd())
import sys
sys.path.append('C:\\Users\\theal\\PycharmProjects')
sys.path.append('C:\\Users\\theal\\PycharmProjects\\stockgym')

C:\Users\theal\PycharmProjects\stockgym\src\agents


In [2]:
import numpy as np
import gym
import random
import time
from stockgym.src.envs.gym_up_and_to_the_right.up_and_to_the_right_env import UpAndToTheRightEnv

In [3]:
import numpy as np
import gym

class RandomActionAgent:
    def __init__(self, env):
        self.env = env

    def get_action(self):
        valid_actions = self.env.controller.get_valid_actions()
        action = np.random.choice(valid_actions)
        return action

    def run_episode(self):
        state = self.env.reset()
        done = False
        total_reward = 0

        while not done:
            action = self.get_action()
            next_state, reward, done, info = self.env.step(action)
            total_reward += reward
            state = next_state

        return total_reward

# Create the environment
env = UpAndToTheRightEnv(state_type="Basic", 
                         reward_type="FinalOnly",
                         num_prev_obvs=5,
                         price_movement_type="Linear",
                         offset_scaling=1.0,
                         scale=1.0,
                         slope=1.0,
                         noise=0.1,
                         starting_price=100,
                         num_steps=100,
                         multiple_units=True,
                         render=False)

# Create and run the agent
agent = RandomActionAgent(env)

for i in range(10):
    total_reward = agent.run_episode()
    print(f"Total reward from the episode {i + 1}: {total_reward}")

Total reward from the episode 1: -3.804891865945015
Total reward from the episode 2: 88.22338393749439
Total reward from the episode 3: 32.347636230038795
Total reward from the episode 4: -65.37357680711635
Total reward from the episode 5: -14.93837980777743
Total reward from the episode 6: 43.17167067580339
Total reward from the episode 7: -0.2670904896128574
Total reward from the episode 8: 25.358572756398978
Total reward from the episode 9: -48.960819614109354
Total reward from the episode 10: -6.530646958437012


In [ ]:
class QLearningAgent:
    def __init__(self, env, learning_rate=.1, discount_factor=0.999, exploration_rate=1, exploration_decay=.99):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.q_table = {}  # Initialize an empty Q-table

    def get_action(self, state):
        state_key = self.state_to_key(state)
        valid_actions = self.env.controller.get_valid_actions()
        self.initialize_state_in_q_table(state_key)

        if random.uniform(0, 1) < self.exploration_rate:
            return random.choice(valid_actions)  # Explore
        else:
            # Exploit by selecting the best valid action based on Q-values
            q_values = np.array([self.q_table[state_key][a] for a in range(self.env.action_space.n)])
            # Mask q_values of invalid actions with a very low number
            mask = np.ones(len(q_values)) * -np.inf
            mask[valid_actions] = 0
            q_values_masked = q_values + mask
            return np.argmax(q_values_masked)  # Exploit

    def update_q_table(self, state, action, reward, next_state):
        state_key = self.state_to_key(state)
        next_state_key = self.state_to_key(next_state)

        # Ensure both the current and next states are in the Q-table
        self.initialize_state_in_q_table(state_key)
        self.initialize_state_in_q_table(next_state_key)

        # Find the best next action from Q-table
        best_next_action = np.argmax(self.q_table[next_state_key])
        td_target = reward + self.discount_factor * self.q_table[next_state_key][best_next_action]
        td_error = td_target - self.q_table[state_key][action]
        self.q_table[state_key][action] += self.learning_rate * td_error


    def initialize_state_in_q_table(self, state_key):
        if state_key not in self.q_table:
            self.q_table[state_key] = [0 for _ in range(self.env.action_space.n)]


    def state_to_key(self, state):
        return tuple(state)


    def run_episode(self):
        state = self.env.reset()
        done = False
        total_reward = 0

        while not done:
            action = self.get_action(state)
            next_state, reward, done, info = self.env.step(action)
            self.update_q_table(state, action, reward, next_state)

            total_reward += reward
            state = next_state

        self.exploration_rate *= self.exploration_decay
        return total_reward
    
    def run_episode_w_render(self):
        state = self.env.reset()
        done = False
        total_reward = 0

        while not done:
            action = self.get_action(state)
            next_state, reward, done, info = self.env.step(action)
            self.update_q_table(state, action, reward, next_state)

            total_reward += reward
            state = next_state
            print(action)
            # self.env.render()

        self.exploration_rate *= self.exploration_decay
        return total_reward

In [22]:
env = UpAndToTheRightEnv(state_type="Basic", 
                         reward_type="FinalOnly",
                         num_prev_obvs=5,
                         price_movement_type="Linear",
                         offset_scaling=1.0,
                         scale=1.0,
                         slope=1.0,
                         noise=0.1,
                         starting_price=100,
                         num_steps=100,
                         multiple_units=True,
                         render=False)

agent = QLearningAgent(env)

In [24]:
for i in range(1000):
    total_reward = agent.run_episode()
    print(f"Total reward from the episode {i + 1}: {total_reward}")

Total reward from the episode 1: 48.11572484536231
Total reward from the episode 2: -2.221685932348245
Total reward from the episode 3: 30.462576475029252
Total reward from the episode 4: 50.02972052634418
Total reward from the episode 5: 27.733330484331823
Total reward from the episode 6: 40.49244174715189
Total reward from the episode 7: 70.56564339192548
Total reward from the episode 8: 48.7957183641452
Total reward from the episode 9: 104.16502270922689
Total reward from the episode 10: 125.29734788840882
Total reward from the episode 11: 79.79196908271523
Total reward from the episode 12: 31.163201950861538
Total reward from the episode 13: -18.97784320493875
Total reward from the episode 14: 33.497727864246706
Total reward from the episode 15: 120.7599175378731
Total reward from the episode 16: 66.12031630605188
Total reward from the episode 17: 49.411374810048855
Total reward from the episode 18: -4.417928160230966
Total reward from the episode 19: -14.851621876442369
Total rewa

Total reward from the episode 160: 51.881466546170024
Total reward from the episode 161: -34.25672023229709
Total reward from the episode 162: 32.29681813313984
Total reward from the episode 163: -55.890593285804314
Total reward from the episode 164: 29.798670034313876
Total reward from the episode 165: 46.59910614183031
Total reward from the episode 166: 47.65779170132878
Total reward from the episode 167: -42.832106450162286
Total reward from the episode 168: 18.644996964688577
Total reward from the episode 169: 15.936550571517154
Total reward from the episode 170: 26.799731465577814
Total reward from the episode 171: 254.41622267646966
Total reward from the episode 172: 114.89862249948648
Total reward from the episode 173: 166.1079553628469
Total reward from the episode 174: 145.27604225723053
Total reward from the episode 175: 51.62440371289143
Total reward from the episode 176: 220.03547866872984
Total reward from the episode 177: 42.19531676614183
Total reward from the episode 17

Total reward from the episode 315: 289.23139328825397
Total reward from the episode 316: 213.00011779282008
Total reward from the episode 317: 89.15001998461517
Total reward from the episode 318: 85.62361852204512
Total reward from the episode 319: 78.20350178916638
Total reward from the episode 320: 36.369082428451016
Total reward from the episode 321: 116.16058135206154
Total reward from the episode 322: 81.94004847757274
Total reward from the episode 323: -11.811136872921292
Total reward from the episode 324: 18.447863372008023
Total reward from the episode 325: 135.2254284013363
Total reward from the episode 326: 62.665370140879986
Total reward from the episode 327: 12.845934419346879
Total reward from the episode 328: 97.40074116896679
Total reward from the episode 329: 122.78629374062236
Total reward from the episode 330: 132.91989212979337
Total reward from the episode 331: 177.93402934923773
Total reward from the episode 332: 109.44310584125388
Total reward from the episode 333

Total reward from the episode 471: 234.1516510908868
Total reward from the episode 472: 91.4553183980711
Total reward from the episode 473: 190.8004497584746
Total reward from the episode 474: 68.40532062485285
Total reward from the episode 475: 242.91398432576227
Total reward from the episode 476: 206.27374645328214
Total reward from the episode 477: 295.8233313208611
Total reward from the episode 478: 153.61423014952578
Total reward from the episode 479: 188.18280440502042
Total reward from the episode 480: 180.32291810629866
Total reward from the episode 481: 159.5247382176124
Total reward from the episode 482: 206.3826326318014
Total reward from the episode 483: 158.85225766045787
Total reward from the episode 484: 152.69501504651424
Total reward from the episode 485: 68.68517592129908
Total reward from the episode 486: 224.93526381978384
Total reward from the episode 487: 98.52380266108457
Total reward from the episode 488: 192.8046869956358
Total reward from the episode 489: 146.

Total reward from the episode 625: 188.6987920120264
Total reward from the episode 626: 248.67294964921783
Total reward from the episode 627: 84.58760195616537
Total reward from the episode 628: 172.68917729970477
Total reward from the episode 629: 44.959328667048716
Total reward from the episode 630: 340.3888317072912
Total reward from the episode 631: 355.27170623611767
Total reward from the episode 632: 348.03688437192466
Total reward from the episode 633: 106.91859897743782
Total reward from the episode 634: 208.1245979148634
Total reward from the episode 635: 157.68374370234096
Total reward from the episode 636: 247.88385531491156
Total reward from the episode 637: 180.96122190488865
Total reward from the episode 638: 177.79855545687587
Total reward from the episode 639: 295.82718125478925
Total reward from the episode 640: 790.8272211105609
Total reward from the episode 641: 205.1687810337337
Total reward from the episode 642: 181.9345295571763
Total reward from the episode 643: 

Total reward from the episode 783: 406.0051660422978
Total reward from the episode 784: 425.98769111808406
Total reward from the episode 785: 74.51764583516967
Total reward from the episode 786: 374.31084975920294
Total reward from the episode 787: 295.1327399027431
Total reward from the episode 788: 169.89753604316442
Total reward from the episode 789: 143.71645080245418
Total reward from the episode 790: 170.18200184579484
Total reward from the episode 791: 332.6173276290833
Total reward from the episode 792: 220.3799559384874
Total reward from the episode 793: 283.3608093518836
Total reward from the episode 794: 309.4493510092018
Total reward from the episode 795: 445.4446130251562
Total reward from the episode 796: 551.3903082770777
Total reward from the episode 797: 576.8127653524891
Total reward from the episode 798: 634.8357814963654
Total reward from the episode 799: 197.31464468864414
Total reward from the episode 800: 344.4544890053499
Total reward from the episode 801: 278.3

Total reward from the episode 939: 548.8485165633516
Total reward from the episode 940: 367.0766199734409
Total reward from the episode 941: 357.6213389957652
Total reward from the episode 942: 687.5108176891276
Total reward from the episode 943: 632.6217866920335
Total reward from the episode 944: 465.6282701472396
Total reward from the episode 945: 294.23713871411223
Total reward from the episode 946: 275.2574269046589
Total reward from the episode 947: 562.4181875602067
Total reward from the episode 948: 409.48048351759263
Total reward from the episode 949: 631.8298960313505
Total reward from the episode 950: 551.2856943115844
Total reward from the episode 951: 395.5331223652724
Total reward from the episode 952: 256.83995199489925
Total reward from the episode 953: 278.095829162266
Total reward from the episode 954: 379.9813947649913
Total reward from the episode 955: 331.53418984822565
Total reward from the episode 956: 209.13976378636644
Total reward from the episode 957: -117.19

In [ ]:
total_reward = agent.run_episode_w_render()

In [8]:
print(total_reward)

3374.5160847884927


In [10]:
agent.q_table

{(0.01,): [366.2202456951612, 11504.881429360406, 1087.6490088514588, 0, 0],
 (0.01, 1.0): [29.64955190324724,
  780.5633847142296,
  11716.90688103198,
  249.8519792659809,
  46.93757576578879],
 (0.01, 0.505, 1.0): [21.82343006816371,
  948.9885988275026,
  11929.976707023152,
  215.71586494251707,
  4.250745122399426],
 (0.01, 0.33999999999999997, 0.67, 1.0): [50.70541269358468,
  485.4321024852635,
  12143.66914519927,
  743.48219152789,
  15.635010471951222],
 (0.01, 0.2575, 0.505, 0.7525, 1.0): [12359.715360250812,
  1971.0868154992734,
  7437.656008919959,
  12143.66914519927,
  6930.710762133917]}